# 15 min

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("Full_data_15m.csv")

In [2]:
df

,Date_15m,datetime,text,username,kk08_pos_score,kk08_score_diff,Elkulako_label,Sentiment_vader,pred,analysis,...,collapse,ATH,grow,rise,fall,date,open,close,Return,Return_sign
0,2022-01-01 00:15:00,"['2022-01-01 00:00:00', '2022-01-01 00:00:00',...",Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,CryptoNerdApp\nTrendSpider\nMadStudentScie1\nW...,0.816322,0.632644,0.342175,0.161173,11,4,...,0,9,6,7,4,2022-01-01 00:15:00,46332.52,46375.42,0.000926,1
1,2022-01-01 00:30:00,"['2022-01-01 00:15:01', '2022-01-01 00:15:05',...",The current price of Bitcoin is $46453.00.\nTh...,big_blockers\nbmurphypointman\nButterLemonBot\...,0.716851,0.433702,0.305638,0.136947,9,42,...,1,8,1,6,38,2022-01-01 00:30:00,46375.42,46610.81,0.005076,1
2,2022-01-01 00:45:00,"['2022-01-01 00:30:00', '2022-01-01 00:30:01',...",“Here's to a bright New Year and a fond farewe...,bitcoiva\ncryptozone007\nlinuxadvocate\nbakedn...,0.725069,0.450138,0.438272,0.172915,9,15,...,0,6,2,5,17,2022-01-01 00:45:00,46610.81,46656.13,0.000972,1
3,2022-01-01 01:00:00,"['2022-01-01 00:45:02', '2022-01-01 00:45:04',...","#Bitcoin $BTC \nPrice: $46,630.10\n\nAreas of ...",DCA_Cryptoz\nbidcoindad\ncoinok\nbmurphypointm...,0.756357,0.512714,0.381818,0.178866,5,11,...,1,5,1,2,6,2022-01-01 01:00:00,46656.14,46766.99,0.002376,1
4,2022-01-01 01:15:00,"['2022-01-01 01:00:00', '2022-01-01 01:00:00',...",Today's Cryptocurrency Fear And Greed Index: 2...,FearAndGreedBot\nOfficialOverbit\nBTC_AI_bot\n...,0.770306,0.540612,0.343195,0.143037,6,13,...,0,6,8,4,13,2022-01-01 01:15:00,46766.99,46815.33,0.001034,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,2022-01-31 23:00:00,"['2022-01-31 22:45:00', '2022-01-31 22:45:01',...",NFL Stars and Politicians' Bet in Bitcoin Turn...,got_eth\nzero_2_nothing\nNewsCryptoBot\nJamesW...,0.771425,0.542850,0.209877,0.167436,1,4,...,1,10,3,3,8,2022-01-31 23:00:00,38412.45,38451.36,0.001013,1
2972,2022-01-31 23:15:00,"['2022-01-31 23:00:00', '2022-01-31 23:00:00',...",Top #Cryptocurrency RANK DOWN (#crypto #bitcoi...,CryptoTopCharts\nBTC_AI_bot\ncryptonewsleak\nt...,0.779117,0.558234,0.263158,0.161682,4,4,...,0,5,5,5,3,2022-01-31 23:15:00,38451.36,38412.23,-0.001018,0
2973,2022-01-31 23:30:00,"['2022-01-31 23:15:00', '2022-01-31 23:15:00',...",Why are people buying #Bitcoin? Why are they a...,ForwardGroupMS\nThomasBakerMS\nbig_blockers\nb...,0.783670,0.567341,0.278592,0.161257,5,6,...,0,6,5,5,3,2022-01-31 23:30:00,38412.23,38369.11,-0.001123,0
2974,2022-01-31 23:45:00,"['2022-01-31 23:30:01', '2022-01-31 23:30:02',...","Bitcoin On Its Way To $500,000? Anthony Scaram...",OutlawBuckeye\nSifHomes\nitvc_\nRaulGlzC\nTheM...,0.769362,0.538723,0.240642,0.110640,3,4,...,1,4,1,8,6,2022-01-31 23:45:00,38369.12,38466.90,0.002548,1


# Cross-validation

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy
import pandas as pd

def cv_and_predict_valid(
    df_x,
    df_y,
    model,
    train_share=0.9,
    tree = False,
    grid_search=False,
    grid_parameters=None
):
    
    feature_importances = []

    split_index = int(train_share * len(df_x)) 
    X_train, X_valid = df_x[:split_index], df_x[split_index:]
    y_train, y_valid = df_y[:split_index], df_y[split_index:]
    
    
    # if scaler:
    #     scaler = MinMaxScaler()
    #     X_train["kk08_pos_score"] = scaler.fit_transform(X_train[["kk08_pos_score"]])   
    #     X_valid["kk08_pos_score"] = scaler.transform(X_valid[["kk08_pos_score"]]) 


    if grid_search and grid_parameters:
        grid_search_cv = GridSearchCV(
            estimator=model,
            param_grid=grid_parameters,
            cv=TimeSeriesSplit(n_splits=5),  
            scoring="balanced_accuracy",  
            n_jobs=-1        
        )
        grid_search_cv.fit(X_train, y_train)
        print(grid_search_cv.best_estimator_)
        model = grid_search_cv.best_estimator_  
        print(f"Best parameters from grid search: {grid_search_cv.best_params_}")
    else:
        model = deepcopy(model)
        model.fit(X_train, y_train)
    

    if tree:
        feature_importances = model.feature_importances_

    
    predictions = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, predictions)
    roc_auc = roc_auc_score(y_valid, predictions)
    f1 = f1_score(y_valid, predictions)

    
    pred_df = pd.DataFrame()
    pred_df["target"] = y_valid
    pred_df["prediction"] = predictions

    
    print(f"Accuracy: {accuracy}")
    print(f"ROC-AUC: {roc_auc}")
    print(f"F1-score: {f1}")

    return pred_df, accuracy, roc_auc, f1, feature_importances


# Only kk08

In [4]:
df_x_1 = df[["kk08_pos_score"]]
df_y = df["Return_sign"]

In [5]:
train_share = 0.85
df_x_1_train = df_x_1.iloc[:int(len(df_x_1)*train_share)].reset_index(drop=True)
df_y_train = df_y.iloc[:int(len(df_x_1)*train_share)].reset_index(drop=True)

df_x_1_test = df_x_1.iloc[int(len(df_x_1)*train_share):].reset_index(drop=True)
df_y_test = df_y.iloc[int(len(df_x_1)*train_share):].reset_index(drop=True)

In [6]:
sum(df_y_test)/len(df_y_test)

0.5279642058165548

In [7]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_1_train,df_y_train,dummy_model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [8]:
# testing
dummy_model.fit(df_x_1_train, df_y_train)
predictions = dummy_model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.4720357941834452
ROC-AUC: 0.5
F1-score: 0.0


In [9]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {
    'n_estimators': [10, 30, 50, 100, 200],
    'max_depth': [3, 5, None],
    'min_samples_leaf': [1, 5, 10]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(min_samples_leaf=10, n_estimators=30, random_state=17)
Best parameters from grid search: {'max_depth': None, 'min_samples_leaf': 10, 'n_estimators': 30}
Accuracy: 0.5059288537549407
ROC-AUC: 0.5061554805649294
F1-score: 0.4769874476987448


In [10]:
# testing
model = RandomForestClassifier(min_samples_leaf=10, n_estimators=30,random_state=17)
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4988814317673378
ROC-AUC: 0.5025805285565106
F1-score: 0.4790697674418605


In [11]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500],"max_depth": [1,2,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 6, 'n_estimators': 100}
Accuracy: 0.5217391304347826
ROC-AUC: 0.52212223472066
F1-score: 0.47161572052401746


In [12]:
# testing
model = XGBClassifier(max_depth=6, n_estimators=100, random_state=17)
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.48322147651006714
ROC-AUC: 0.4857418266527431
F1-score: 0.47380410022779046


In [13]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1, 2],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, svm_model, grid_search=True, grid_parameters=grid_params)

SVC(C=2, class_weight='balanced', gamma=1)
Best parameters from grid search: {'C': 2, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'}
Accuracy: 0.466403162055336
ROC-AUC: 0.4672540932383452
F1-score: 0.32160804020100503


In [14]:
# testing
model = SVC(C=2, class_weight='balanced', gamma=1)
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.47874720357941836
ROC-AUC: 0.47974736926660777
F1-score: 0.48337028824833705


In [15]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
ada_model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, ada_model, tree = True, grid_search=True, grid_parameters = grid_params)


AdaBoostClassifier(learning_rate=0.3, n_estimators=20)
Best parameters from grid search: {'learning_rate': 0.3, 'n_estimators': 20}
Accuracy: 0.5059288537549407
ROC-AUC: 0.5078740157480315
F1-score: 0.031007751937984496


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [16]:
# testing
model = AdaBoostClassifier(learning_rate=0.3, n_estimators=20)
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.47651006711409394
ROC-AUC: 0.5032331914209976
F1-score: 0.04878048780487805


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [17]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, nb_model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [18]:
# testing
model = GaussianNB()
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4720357941834452
ROC-AUC: 0.5
F1-score: 0.0


In [19]:
from sklearn.linear_model import LogisticRegression
grid_params = {
    'C': [0.01, 0.1, 1],           
    'penalty': ['l2']  
}
lr_model = LogisticRegression()
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, lr_model, grid_search=True, grid_parameters=grid_params)


LogisticRegression(C=0.01)
Best parameters from grid search: {'C': 0.01, 'penalty': 'l2'}
Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [20]:
# testing
model = LogisticRegression(C=0.01)
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4720357941834452
ROC-AUC: 0.5
F1-score: 0.0


In [21]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5,6,7,8,9,10,15]}
knn_model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, knn_model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier(n_neighbors=3)
Best parameters from grid search: {'n_neighbors': 3}
Accuracy: 0.48616600790513836
ROC-AUC: 0.48631421072365955
F1-score: 0.4672131147540984


In [22]:
# testing
model = KNeighborsClassifier(n_neighbors=3)
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.5100671140939598
ROC-AUC: 0.5116676038236003
F1-score: 0.5100671140939598


# kk08 with lags

In [23]:
df['kk08_pos_score_lag1']=df['kk08_pos_score'].shift(1)
df['kk08_pos_score_lag2']=df['kk08_pos_score'].shift(2)
df['kk08_pos_score_lag3']=df['kk08_pos_score'].shift(3)


In [24]:
df_x_2 = df.dropna().reset_index(drop=True)[["kk08_pos_score", "kk08_pos_score_lag1", "kk08_pos_score_lag2", "kk08_pos_score_lag3"]]
df_y_2 = df["Return_sign"].iloc[3:].reset_index(drop=True)

In [25]:
df_x_2

,kk08_pos_score,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3
0,0.756357,0.725069,0.716851,0.816322
1,0.770306,0.756357,0.725069,0.716851
2,0.765516,0.770306,0.756357,0.725069
3,0.768058,0.765516,0.770306,0.756357
4,0.805194,0.768058,0.765516,0.770306
...,...,...,...,...
2968,0.771425,0.780470,0.790736,0.812888
2969,0.779117,0.771425,0.780470,0.790736
2970,0.783670,0.779117,0.771425,0.780470
2971,0.769362,0.783670,0.779117,0.771425


In [26]:
df_y_2

0       1
1       1
2       0
3       1
4       1
       ..
2968    1
2969    0
2970    0
2971    1
2972    1
Name: Return_sign, Length: 2973, dtype: int64

In [27]:
train_share = 0.85
df_x_2_train = df_x_2.iloc[:int(len(df_x_2)*train_share)].reset_index(drop=True)
df_y_2_train = df_y_2.iloc[:int(len(df_x_2)*train_share)].reset_index(drop=True)

df_x_2_test = df_x_2.iloc[int(len(df_x_2)*train_share):].reset_index(drop=True)
df_y_2_test = df_y_2.iloc[int(len(df_x_2)*train_share):].reset_index(drop=True)

In [28]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_2_train,df_y_2_train,dummy_model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [29]:
# testing
dummy_model.fit(df_x_2_train, df_y_2_train)
predictions = dummy_model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.4730941704035874
ROC-AUC: 0.5
F1-score: 0.0


In [30]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {
    'n_estimators': [10, 30, 50, 100, 200],
    'max_depth': [3, 5, None],
    'min_samples_leaf': [1, 5, 10]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(min_samples_leaf=5, n_estimators=10, random_state=17)
Best parameters from grid search: {'max_depth': None, 'min_samples_leaf': 5, 'n_estimators': 10}
Accuracy: 0.4782608695652174
ROC-AUC: 0.4787526559180103
F1-score: 0.40540540540540543


In [31]:
# testing
model = RandomForestClassifier(min_samples_leaf=5, n_estimators=10, random_state=17)

model.fit(df_x_2_train, df_y_2_train)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.48878923766816146
ROC-AUC: 0.4935968538872643
F1-score: 0.45454545454545453


In [32]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500],"max_depth": [1,2,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 3, 'n_estimators': 500}
Accuracy: 0.5019762845849802
ROC-AUC: 0.5022497187851518
F1-score: 0.4661016949152542


In [168]:
# testing
model = XGBClassifier(max_depth=3, n_estimators=500, random_state=17)

model.fit(df_x_2_train, df_y_2_train)
print(model.feature_importances_)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)

print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# not bad

[0.25900617 0.24520864 0.2514413  0.24434394]
Accuracy: 0.5381165919282511
ROC-AUC: 0.5391953211656751
F1-score: 0.5422222222222223


In [34]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1, 2],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, svm_model, grid_search=True, grid_parameters=grid_params)

SVC(C=2, class_weight='balanced', gamma=1)
Best parameters from grid search: {'C': 2, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'}
Accuracy: 0.49407114624505927
ROC-AUC: 0.4949381327334083
F1-score: 0.35353535353535354


In [35]:
# testing
model = SVC(C=2, class_weight='balanced', gamma=1)

model.fit(df_x_2_train, df_y_2_train)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4820627802690583
ROC-AUC: 0.48116365836442476
F1-score: 0.5032258064516129


In [36]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
ada_model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, ada_model, tree = True, grid_search=True, grid_parameters = grid_params)


AdaBoostClassifier(learning_rate=0.5, n_estimators=20)
Best parameters from grid search: {'learning_rate': 0.5, 'n_estimators': 20}
Accuracy: 0.5019762845849802
ROC-AUC: 0.5032495938007749
F1-score: 0.26744186046511625


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [37]:
# testing
model = AdaBoostClassifier(learning_rate=0.5, n_estimators=20)
model.fit(df_x_2_train, df_y_2_train)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.48878923766816146
ROC-AUC: 0.5119895129575476
F1-score: 0.14285714285714285


In [38]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, nb_model)


Accuracy: 0.4901185770750988
ROC-AUC: 0.4920634920634921
F1-score: 0.0


In [39]:
# testing
model = GaussianNB()

model.fit(df_x_2_train, df_y_2_train)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.47533632286995514
ROC-AUC: 0.4987395381667843
F1-score: 0.11363636363636363


In [40]:
from sklearn.linear_model import LogisticRegression
grid_params = {
    'C': [0.01, 0.1, 1],           
    'penalty': ['l2']  
}
lr_model = LogisticRegression()
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, lr_model, grid_search=True, grid_parameters=grid_params)


LogisticRegression(C=1)
Best parameters from grid search: {'C': 1, 'penalty': 'l2'}
Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [41]:
# testing
model = LogisticRegression(C=1)

model.fit(df_x_2_train, df_y_2_train)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.47533632286995514
ROC-AUC: 0.502127659574468
F1-score: 0.00847457627118644


In [42]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
knn_model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, knn_model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier(n_neighbors=50)
Best parameters from grid search: {'n_neighbors': 50}
Accuracy: 0.5375494071146245
ROC-AUC: 0.5386514185726784
F1-score: 0.36065573770491804


In [43]:
# testing
model = KNeighborsClassifier(n_neighbors=50)

model.fit(df_x_2_train, df_y_2_train)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.5179372197309418
ROC-AUC: 0.5275486538267622
F1-score: 0.43271767810026385


# Only vader

In [44]:
df_x_3 = df[["Sentiment_vader"]]
df_y = df["Return_sign"]

In [45]:
train_share = 0.85
df_x_3_train = df_x_3.iloc[:int(len(df_x_3)*train_share)].reset_index(drop=True)
df_y_train = df_y.iloc[:int(len(df_x_3)*train_share)].reset_index(drop=True)

df_x_3_test = df_x_3.iloc[int(len(df_x_3)*train_share):].reset_index(drop=True)
df_y_test = df_y.iloc[int(len(df_x_3)*train_share):].reset_index(drop=True)

In [46]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_3_train,df_y_train,dummy_model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [47]:
# testing
model = DummyClassifier(strategy='most_frequent')

model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4720357941834452
ROC-AUC: 0.5
F1-score: 0.0


In [48]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {'n_estimators': [5, 10, 20, 50,100]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(n_estimators=20, random_state=17)
Best parameters from grid search: {'n_estimators': 20}
Accuracy: 0.525691699604743
ROC-AUC: 0.5258405199350081
F1-score: 0.5081967213114754


In [49]:
# testing
model = RandomForestClassifier(n_estimators=20,random_state=17)
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4697986577181208
ROC-AUC: 0.47127279299542135
F1-score: 0.4697986577181208


In [50]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500],"max_depth": [1,2,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 1, 'n_estimators': 300}
Accuracy: 0.4901185770750988
ROC-AUC: 0.4909073865766779
F1-score: 0.3645320197044335


In [51]:
# testing
model = XGBClassifier(random_state=17, max_depth = 1, n_estimators = 300)
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.5055928411633109
ROC-AUC: 0.5167182102980159
F1-score: 0.40431266846361186


In [52]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1, 2],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, svm_model, grid_search=True, grid_parameters=grid_params)

SVC(C=2, class_weight='balanced', gamma=1)
Best parameters from grid search: {'C': 2, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'}
Accuracy: 0.5019762845849802
ROC-AUC: 0.5
F1-score: 0.6684210526315789


In [53]:
# testing
model = SVC(C=2, class_weight='balanced', gamma=1, kernel='poly')
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4720357941834452
ROC-AUC: 0.5
F1-score: 0.0


In [54]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
ada_model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, ada_model, tree = True, grid_search=True, grid_parameters = grid_params)


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=0.5, n_estimators=500)
Best parameters from grid search: {'learning_rate': 0.5, 'n_estimators': 500}
Accuracy: 0.49407114624505927
ROC-AUC: 0.4946881639795026
F1-score: 0.40186915887850466


In [55]:
# testing
model = AdaBoostClassifier(learning_rate=0.5, n_estimators=500)
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.4988814317673378
ROC-AUC: 0.5010743834846172
F1-score: 0.49321266968325794


In [56]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, nb_model)


Accuracy: 0.5019762845849802
ROC-AUC: 0.5039370078740157
F1-score: 0.015625


In [57]:
# testing
model = GaussianNB()
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4720357941834452
ROC-AUC: 0.4994979516427022
F1-score: 0.016666666666666666


In [58]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, lr_model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [59]:
# testing
model = LogisticRegression()
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4720357941834452
ROC-AUC: 0.5
F1-score: 0.0


In [60]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
knn_model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, knn_model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier(n_neighbors=50)
Best parameters from grid search: {'n_neighbors': 50}
Accuracy: 0.48221343873517786
ROC-AUC: 0.4827834020747407
F1-score: 0.39631336405529954


In [61]:
# testing
model = KNeighborsClassifier(n_neighbors=50)
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.48769574944071586
ROC-AUC: 0.49550164671861197
F1-score: 0.42317380352644834


# Vader with lags

In [62]:
df

,Date_15m,datetime,text,username,kk08_pos_score,kk08_score_diff,Elkulako_label,Sentiment_vader,pred,analysis,...,rise,fall,date,open,close,Return,Return_sign,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3
0,2022-01-01 00:15:00,"['2022-01-01 00:00:00', '2022-01-01 00:00:00',...",Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,CryptoNerdApp\nTrendSpider\nMadStudentScie1\nW...,0.816322,0.632644,0.342175,0.161173,11,4,...,7,4,2022-01-01 00:15:00,46332.52,46375.42,0.000926,1,NaN,NaN,NaN
1,2022-01-01 00:30:00,"['2022-01-01 00:15:01', '2022-01-01 00:15:05',...",The current price of Bitcoin is $46453.00.\nTh...,big_blockers\nbmurphypointman\nButterLemonBot\...,0.716851,0.433702,0.305638,0.136947,9,42,...,6,38,2022-01-01 00:30:00,46375.42,46610.81,0.005076,1,0.816322,NaN,NaN
2,2022-01-01 00:45:00,"['2022-01-01 00:30:00', '2022-01-01 00:30:01',...",“Here's to a bright New Year and a fond farewe...,bitcoiva\ncryptozone007\nlinuxadvocate\nbakedn...,0.725069,0.450138,0.438272,0.172915,9,15,...,5,17,2022-01-01 00:45:00,46610.81,46656.13,0.000972,1,0.716851,0.816322,NaN
3,2022-01-01 01:00:00,"['2022-01-01 00:45:02', '2022-01-01 00:45:04',...","#Bitcoin $BTC \nPrice: $46,630.10\n\nAreas of ...",DCA_Cryptoz\nbidcoindad\ncoinok\nbmurphypointm...,0.756357,0.512714,0.381818,0.178866,5,11,...,2,6,2022-01-01 01:00:00,46656.14,46766.99,0.002376,1,0.725069,0.716851,0.816322
4,2022-01-01 01:15:00,"['2022-01-01 01:00:00', '2022-01-01 01:00:00',...",Today's Cryptocurrency Fear And Greed Index: 2...,FearAndGreedBot\nOfficialOverbit\nBTC_AI_bot\n...,0.770306,0.540612,0.343195,0.143037,6,13,...,4,13,2022-01-01 01:15:00,46766.99,46815.33,0.001034,1,0.756357,0.725069,0.716851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,2022-01-31 23:00:00,"['2022-01-31 22:45:00', '2022-01-31 22:45:01',...",NFL Stars and Politicians' Bet in Bitcoin Turn...,got_eth\nzero_2_nothing\nNewsCryptoBot\nJamesW...,0.771425,0.542850,0.209877,0.167436,1,4,...,3,8,2022-01-31 23:00:00,38412.45,38451.36,0.001013,1,0.780470,0.790736,0.812888
2972,2022-01-31 23:15:00,"['2022-01-31 23:00:00', '2022-01-31 23:00:00',...",Top #Cryptocurrency RANK DOWN (#crypto #bitcoi...,CryptoTopCharts\nBTC_AI_bot\ncryptonewsleak\nt...,0.779117,0.558234,0.263158,0.161682,4,4,...,5,3,2022-01-31 23:15:00,38451.36,38412.23,-0.001018,0,0.771425,0.780470,0.790736
2973,2022-01-31 23:30:00,"['2022-01-31 23:15:00', '2022-01-31 23:15:00',...",Why are people buying #Bitcoin? Why are they a...,ForwardGroupMS\nThomasBakerMS\nbig_blockers\nb...,0.783670,0.567341,0.278592,0.161257,5,6,...,5,3,2022-01-31 23:30:00,38412.23,38369.11,-0.001123,0,0.779117,0.771425,0.780470
2974,2022-01-31 23:45:00,"['2022-01-31 23:30:01', '2022-01-31 23:30:02',...","Bitcoin On Its Way To $500,000? Anthony Scaram...",OutlawBuckeye\nSifHomes\nitvc_\nRaulGlzC\nTheM...,0.769362,0.538723,0.240642,0.110640,3,4,...,8,6,2022-01-31 23:45:00,38369.12,38466.90,0.002548,1,0.783670,0.779117,0.771425


In [63]:
df['Sentiment_vader_lag1']=df['Sentiment_vader'].shift(1)
df['Sentiment_vader_lag2']=df['Sentiment_vader'].shift(2)
df['Sentiment_vader_lag3']=df['Sentiment_vader'].shift(3)


In [64]:
df_x_4 = df.dropna().reset_index(drop=True)[["Sentiment_vader", "Sentiment_vader_lag1", "Sentiment_vader_lag2", "Sentiment_vader_lag3"]]
df_y_4 = df["Return_sign"].iloc[3:].reset_index(drop=True)

In [65]:
df_x_4

,Sentiment_vader,Sentiment_vader_lag1,Sentiment_vader_lag2,Sentiment_vader_lag3
0,0.178866,0.172915,0.136947,0.161173
1,0.143037,0.178866,0.172915,0.136947
2,0.168293,0.143037,0.178866,0.172915
3,0.201061,0.168293,0.143037,0.178866
4,0.199579,0.201061,0.168293,0.143037
...,...,...,...,...
2968,0.167436,0.171507,0.120725,0.163388
2969,0.161682,0.167436,0.171507,0.120725
2970,0.161257,0.161682,0.167436,0.171507
2971,0.110640,0.161257,0.161682,0.167436


In [66]:
df_y_4

0       1
1       1
2       0
3       1
4       1
       ..
2968    1
2969    0
2970    0
2971    1
2972    1
Name: Return_sign, Length: 2973, dtype: int64

In [67]:
train_share = 0.85
df_x_4_train = df_x_4.iloc[:int(len(df_x_4)*train_share)].reset_index(drop=True)
df_y_4_train = df_y_4.iloc[:int(len(df_x_4)*train_share)].reset_index(drop=True)

df_x_4_test = df_x_4.iloc[int(len(df_x_4)*train_share):].reset_index(drop=True)
df_y_4_test = df_y_4.iloc[int(len(df_x_4)*train_share):].reset_index(drop=True)

In [68]:
sum(df_y_4_test)/len(df_y_4_test)

0.5269058295964125

In [69]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_4_train,df_y_4_train,dummy_model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [70]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {'n_estimators': [5, 10, 20, 50, 100, 300]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(n_estimators=5, random_state=17)
Best parameters from grid search: {'n_estimators': 5}
Accuracy: 0.525691699604743
ROC-AUC: 0.526121734783152
F1-score: 0.4690265486725664


In [169]:
# testing
model = RandomForestClassifier(n_estimators=5, random_state=17)
model.fit(df_x_4_train, df_y_4_train)
print(model.feature_importances_)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# not bad

[0.23936391 0.25814306 0.25014016 0.25235287]
Accuracy: 0.5426008968609866
ROC-AUC: 0.5456287183624079
F1-score: 0.5299539170506913


In [72]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [100, 300, 500],"max_depth": [1,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 1, 'n_estimators': 100}
Accuracy: 0.5059288537549407
ROC-AUC: 0.5069366329208849
F1-score: 0.3386243386243386


In [73]:
# testing
model = XGBClassifier(random_state=17, max_depth = 1, n_estimators= 100)
model.fit(df_x_4_train, df_y_4_train)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

Accuracy: 0.4618834080717489
ROC-AUC: 0.4692749823535344
F1-score: 0.3939393939393939


In [74]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, svm_model, grid_search=True, grid_parameters=grid_params)
# too bad

SVC(C=0.1, class_weight='balanced', gamma=1, kernel='poly')
Best parameters from grid search: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'poly'}
Accuracy: 0.5019762845849802
ROC-AUC: 0.5
F1-score: 0.6684210526315789


In [75]:
# testing
model = SVC(C=0.1, class_weight='balanced', gamma=1, kernel='poly')
model.fit(df_x_4_train, df_y_4_train)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4730941704035874
ROC-AUC: 0.5
F1-score: 0.0


In [76]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, model, tree = True, grid_search=True, grid_parameters = grid_params)


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=0.5)
Best parameters from grid search: {'learning_rate': 0.5, 'n_estimators': 50}
Accuracy: 0.5296442687747036
ROC-AUC: 0.5309961254843145
F1-score: 0.2874251497005988


In [77]:
# testing
model = model = AdaBoostClassifier(learning_rate=0.5, n_estimators=50)
model.fit(df_x_4_train, df_y_4_train)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.48654708520179374
ROC-AUC: 0.4943732983765252
F1-score: 0.4173027989821883


In [78]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, model)


Accuracy: 0.44664031620553357
ROC-AUC: 0.4466629171353581
F1-score: 0.4444444444444444


In [79]:
# testing
model = model = GaussianNB()
model.fit(df_x_4_train, df_y_4_train)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.4977578475336323
ROC-AUC: 0.5006554401532721
F1-score: 0.4838709677419355


In [80]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=0.5)
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [81]:
# testing
model = model = LogisticRegression(C=0.5)
model.fit(df_x_4_train, df_y_4_train)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4730941704035874
ROC-AUC: 0.5
F1-score: 0.0


In [82]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier(n_neighbors=50)
Best parameters from grid search: {'n_neighbors': 50}
Accuracy: 0.5217391304347826
ROC-AUC: 0.5225909261342332
F1-score: 0.39195979899497485


In [83]:
# testing
model = model = KNeighborsClassifier(n_neighbors=50)
model.fit(df_x_4_train, df_y_4_train)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# not bad

Accuracy: 0.531390134529148
ROC-AUC: 0.5403146112735707
F1-score: 0.45714285714285713


# Only elkulako

In [84]:
df

,Date_15m,datetime,text,username,kk08_pos_score,kk08_score_diff,Elkulako_label,Sentiment_vader,pred,analysis,...,open,close,Return,Return_sign,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3,Sentiment_vader_lag1,Sentiment_vader_lag2,Sentiment_vader_lag3
0,2022-01-01 00:15:00,"['2022-01-01 00:00:00', '2022-01-01 00:00:00',...",Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,CryptoNerdApp\nTrendSpider\nMadStudentScie1\nW...,0.816322,0.632644,0.342175,0.161173,11,4,...,46332.52,46375.42,0.000926,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01 00:30:00,"['2022-01-01 00:15:01', '2022-01-01 00:15:05',...",The current price of Bitcoin is $46453.00.\nTh...,big_blockers\nbmurphypointman\nButterLemonBot\...,0.716851,0.433702,0.305638,0.136947,9,42,...,46375.42,46610.81,0.005076,1,0.816322,NaN,NaN,0.161173,NaN,NaN
2,2022-01-01 00:45:00,"['2022-01-01 00:30:00', '2022-01-01 00:30:01',...",“Here's to a bright New Year and a fond farewe...,bitcoiva\ncryptozone007\nlinuxadvocate\nbakedn...,0.725069,0.450138,0.438272,0.172915,9,15,...,46610.81,46656.13,0.000972,1,0.716851,0.816322,NaN,0.136947,0.161173,NaN
3,2022-01-01 01:00:00,"['2022-01-01 00:45:02', '2022-01-01 00:45:04',...","#Bitcoin $BTC \nPrice: $46,630.10\n\nAreas of ...",DCA_Cryptoz\nbidcoindad\ncoinok\nbmurphypointm...,0.756357,0.512714,0.381818,0.178866,5,11,...,46656.14,46766.99,0.002376,1,0.725069,0.716851,0.816322,0.172915,0.136947,0.161173
4,2022-01-01 01:15:00,"['2022-01-01 01:00:00', '2022-01-01 01:00:00',...",Today's Cryptocurrency Fear And Greed Index: 2...,FearAndGreedBot\nOfficialOverbit\nBTC_AI_bot\n...,0.770306,0.540612,0.343195,0.143037,6,13,...,46766.99,46815.33,0.001034,1,0.756357,0.725069,0.716851,0.178866,0.172915,0.136947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,2022-01-31 23:00:00,"['2022-01-31 22:45:00', '2022-01-31 22:45:01',...",NFL Stars and Politicians' Bet in Bitcoin Turn...,got_eth\nzero_2_nothing\nNewsCryptoBot\nJamesW...,0.771425,0.542850,0.209877,0.167436,1,4,...,38412.45,38451.36,0.001013,1,0.780470,0.790736,0.812888,0.171507,0.120725,0.163388
2972,2022-01-31 23:15:00,"['2022-01-31 23:00:00', '2022-01-31 23:00:00',...",Top #Cryptocurrency RANK DOWN (#crypto #bitcoi...,CryptoTopCharts\nBTC_AI_bot\ncryptonewsleak\nt...,0.779117,0.558234,0.263158,0.161682,4,4,...,38451.36,38412.23,-0.001018,0,0.771425,0.780470,0.790736,0.167436,0.171507,0.120725
2973,2022-01-31 23:30:00,"['2022-01-31 23:15:00', '2022-01-31 23:15:00',...",Why are people buying #Bitcoin? Why are they a...,ForwardGroupMS\nThomasBakerMS\nbig_blockers\nb...,0.783670,0.567341,0.278592,0.161257,5,6,...,38412.23,38369.11,-0.001123,0,0.779117,0.771425,0.780470,0.161682,0.167436,0.171507
2974,2022-01-31 23:45:00,"['2022-01-31 23:30:01', '2022-01-31 23:30:02',...","Bitcoin On Its Way To $500,000? Anthony Scaram...",OutlawBuckeye\nSifHomes\nitvc_\nRaulGlzC\nTheM...,0.769362,0.538723,0.240642,0.110640,3,4,...,38369.12,38466.90,0.002548,1,0.783670,0.779117,0.771425,0.161257,0.161682,0.167436


In [85]:
df_x_5 = df[["Elkulako_label"]]
df_y = df["Return_sign"]

In [86]:
df_x_5

,Elkulako_label
0,0.342175
1,0.305638
2,0.438272
3,0.381818
4,0.343195
...,...
2971,0.209877
2972,0.263158
2973,0.278592
2974,0.240642


In [87]:
train_share = 0.85
df_x_5_train = df_x_5.iloc[:int(len(df_x_5)*train_share)].reset_index(drop=True)
df_y_train = df_y.iloc[:int(len(df_x_5)*train_share)].reset_index(drop=True)

df_x_5_test = df_x_5.iloc[int(len(df_x_5)*train_share):].reset_index(drop=True)
df_y_test = df_y.iloc[int(len(df_x_5)*train_share):].reset_index(drop=True)

In [88]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_5_train,df_y_train,dummy_model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [89]:
# testing
model = DummyClassifier(strategy='most_frequent')

model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4720357941834452
ROC-AUC: 0.5
F1-score: 0.0


In [90]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {'n_estimators': [5, 10, 20, 50,100]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(n_estimators=50, random_state=17)
Best parameters from grid search: {'n_estimators': 50}
Accuracy: 0.47035573122529645
ROC-AUC: 0.470347456567929
F1-score: 0.47244094488188976


In [171]:
# testing
model = RandomForestClassifier(n_estimators=50, random_state=17)
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

Accuracy: 0.5346756152125279
ROC-AUC: 0.5344706402120653
F1-score: 0.5497835497835498


In [92]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500],"max_depth": [1,2,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=5, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 1, 'n_estimators': 5}
Accuracy: 0.4782608695652174
ROC-AUC: 0.4800649918760155
F1-score: 0.043478260869565216


In [93]:
xgb_model = XGBClassifier(random_state=17, max_depth = 1, n_estimators = 5)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, xgb_model, tree = True)

Accuracy: 0.4782608695652174
ROC-AUC: 0.4800649918760155
F1-score: 0.043478260869565216


In [94]:
# testing
model = XGBClassifier(random_state=17, max_depth = 6, n_estimators = 300)
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.4988814317673378
ROC-AUC: 0.5005723351273195
F1-score: 0.4977578475336323


In [95]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1, 2],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, svm_model, grid_search=True, grid_parameters=grid_params)

SVC(C=0.1, class_weight='balanced', gamma=1, kernel='poly')
Best parameters from grid search: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'poly'}
Accuracy: 0.5019762845849802
ROC-AUC: 0.5
F1-score: 0.6684210526315789


In [96]:
# testing
model = SVC(C=0.1, class_weight='balanced', gamma=1, kernel='poly')
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4720357941834452
ROC-AUC: 0.5
F1-score: 0.0


In [97]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
ada_model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, ada_model, tree = True, grid_search=True, grid_parameters = grid_params)


AdaBoostClassifier(learning_rate=1, n_estimators=5)
Best parameters from grid search: {'learning_rate': 1, 'n_estimators': 5}
Accuracy: 0.4901185770750988
ROC-AUC: 0.49193850768653913
F1-score: 0.058394160583941604


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [98]:
# testing
model = AdaBoostClassifier(learning_rate=0.1, n_estimators=5)
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4720357941834452
ROC-AUC: 0.5
F1-score: 0.0


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [99]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, nb_model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [100]:
# testing
model = GaussianNB()
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4720357941834452
ROC-AUC: 0.5
F1-score: 0.0


In [101]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, lr_model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [102]:
# testing
model = LogisticRegression()
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4720357941834452
ROC-AUC: 0.5
F1-score: 0.0


In [103]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
knn_model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, knn_model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier(n_neighbors=50)
Best parameters from grid search: {'n_neighbors': 50}
Accuracy: 0.4782608695652174
ROC-AUC: 0.47909636295463065
F1-score: 0.34


In [104]:
# testing
model = KNeighborsClassifier(n_neighbors=50)
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.5167785234899329
ROC-AUC: 0.5235460679572657
F1-score: 0.46798029556650245


# Elkulako with lags

In [105]:
df

,Date_15m,datetime,text,username,kk08_pos_score,kk08_score_diff,Elkulako_label,Sentiment_vader,pred,analysis,...,open,close,Return,Return_sign,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3,Sentiment_vader_lag1,Sentiment_vader_lag2,Sentiment_vader_lag3
0,2022-01-01 00:15:00,"['2022-01-01 00:00:00', '2022-01-01 00:00:00',...",Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,CryptoNerdApp\nTrendSpider\nMadStudentScie1\nW...,0.816322,0.632644,0.342175,0.161173,11,4,...,46332.52,46375.42,0.000926,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01 00:30:00,"['2022-01-01 00:15:01', '2022-01-01 00:15:05',...",The current price of Bitcoin is $46453.00.\nTh...,big_blockers\nbmurphypointman\nButterLemonBot\...,0.716851,0.433702,0.305638,0.136947,9,42,...,46375.42,46610.81,0.005076,1,0.816322,NaN,NaN,0.161173,NaN,NaN
2,2022-01-01 00:45:00,"['2022-01-01 00:30:00', '2022-01-01 00:30:01',...",“Here's to a bright New Year and a fond farewe...,bitcoiva\ncryptozone007\nlinuxadvocate\nbakedn...,0.725069,0.450138,0.438272,0.172915,9,15,...,46610.81,46656.13,0.000972,1,0.716851,0.816322,NaN,0.136947,0.161173,NaN
3,2022-01-01 01:00:00,"['2022-01-01 00:45:02', '2022-01-01 00:45:04',...","#Bitcoin $BTC \nPrice: $46,630.10\n\nAreas of ...",DCA_Cryptoz\nbidcoindad\ncoinok\nbmurphypointm...,0.756357,0.512714,0.381818,0.178866,5,11,...,46656.14,46766.99,0.002376,1,0.725069,0.716851,0.816322,0.172915,0.136947,0.161173
4,2022-01-01 01:15:00,"['2022-01-01 01:00:00', '2022-01-01 01:00:00',...",Today's Cryptocurrency Fear And Greed Index: 2...,FearAndGreedBot\nOfficialOverbit\nBTC_AI_bot\n...,0.770306,0.540612,0.343195,0.143037,6,13,...,46766.99,46815.33,0.001034,1,0.756357,0.725069,0.716851,0.178866,0.172915,0.136947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,2022-01-31 23:00:00,"['2022-01-31 22:45:00', '2022-01-31 22:45:01',...",NFL Stars and Politicians' Bet in Bitcoin Turn...,got_eth\nzero_2_nothing\nNewsCryptoBot\nJamesW...,0.771425,0.542850,0.209877,0.167436,1,4,...,38412.45,38451.36,0.001013,1,0.780470,0.790736,0.812888,0.171507,0.120725,0.163388
2972,2022-01-31 23:15:00,"['2022-01-31 23:00:00', '2022-01-31 23:00:00',...",Top #Cryptocurrency RANK DOWN (#crypto #bitcoi...,CryptoTopCharts\nBTC_AI_bot\ncryptonewsleak\nt...,0.779117,0.558234,0.263158,0.161682,4,4,...,38451.36,38412.23,-0.001018,0,0.771425,0.780470,0.790736,0.167436,0.171507,0.120725
2973,2022-01-31 23:30:00,"['2022-01-31 23:15:00', '2022-01-31 23:15:00',...",Why are people buying #Bitcoin? Why are they a...,ForwardGroupMS\nThomasBakerMS\nbig_blockers\nb...,0.783670,0.567341,0.278592,0.161257,5,6,...,38412.23,38369.11,-0.001123,0,0.779117,0.771425,0.780470,0.161682,0.167436,0.171507
2974,2022-01-31 23:45:00,"['2022-01-31 23:30:01', '2022-01-31 23:30:02',...","Bitcoin On Its Way To $500,000? Anthony Scaram...",OutlawBuckeye\nSifHomes\nitvc_\nRaulGlzC\nTheM...,0.769362,0.538723,0.240642,0.110640,3,4,...,38369.12,38466.90,0.002548,1,0.783670,0.779117,0.771425,0.161257,0.161682,0.167436


In [106]:
df['Elkulako_label_lag1']=df['Elkulako_label'].shift(1)
df['Elkulako_label_lag2']=df['Elkulako_label'].shift(2)
df['Elkulako_label_lag3']=df['Elkulako_label'].shift(3)


In [107]:
df_x_6 = df.dropna().reset_index(drop=True)[["Elkulako_label", "Elkulako_label_lag1", "Elkulako_label_lag2", "Elkulako_label_lag3"]]
df_y_6 = df["Return_sign"].iloc[3:].reset_index(drop=True)

In [108]:
df_x_6

,Elkulako_label,Elkulako_label_lag1,Elkulako_label_lag2,Elkulako_label_lag3
0,0.381818,0.438272,0.305638,0.342175
1,0.343195,0.381818,0.438272,0.305638
2,0.358209,0.343195,0.381818,0.438272
3,0.323529,0.358209,0.343195,0.381818
4,0.345455,0.323529,0.358209,0.343195
...,...,...,...,...
2968,0.209877,0.202857,0.214076,0.241458
2969,0.263158,0.209877,0.202857,0.214076
2970,0.278592,0.263158,0.209877,0.202857
2971,0.240642,0.278592,0.263158,0.209877


In [109]:
df_y_6

0       1
1       1
2       0
3       1
4       1
       ..
2968    1
2969    0
2970    0
2971    1
2972    1
Name: Return_sign, Length: 2973, dtype: int64

In [110]:
train_share = 0.85
df_x_6_train = df_x_6.iloc[:int(len(df_x_6)*train_share)].reset_index(drop=True)
df_y_6_train = df_y_6.iloc[:int(len(df_x_6)*train_share)].reset_index(drop=True)

df_x_6_test = df_x_6.iloc[int(len(df_x_6)*train_share):].reset_index(drop=True)
df_y_6_test = df_y_6.iloc[int(len(df_x_6)*train_share):].reset_index(drop=True)

In [111]:
sum(df_y_6_test)/len(df_y_6_test)

0.5269058295964125

In [112]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_6_train,df_y_6_train,dummy_model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [113]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {'n_estimators': [5, 10, 20, 50, 100, 300]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(n_estimators=5, random_state=17)
Best parameters from grid search: {'n_estimators': 5}
Accuracy: 0.48616600790513836
ROC-AUC: 0.4864079490063743
F1-score: 0.453781512605042


In [175]:
# testing
model = RandomForestClassifier(n_estimators=5, random_state=17)
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.4641255605381166
ROC-AUC: 0.46438439044065744
F1-score: 0.4747252747252747


In [115]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [100, 300, 500],"max_depth": [1,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 3, 'n_estimators': 100}
Accuracy: 0.5098814229249012
ROC-AUC: 0.5102799650043744
F1-score: 0.45614035087719296


In [116]:
# testing
model = XGBClassifier(random_state=17, max_depth = 3, n_estimators= 100)
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# not bad

Accuracy: 0.47085201793721976
ROC-AUC: 0.47512352525965523
F1-score: 0.44075829383886256


In [117]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, svm_model, grid_search=True, grid_parameters=grid_params)
# too bad

SVC(C=0.1, class_weight='balanced', gamma=1)
Best parameters from grid search: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'}
Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [118]:
# testing
model = SVC(C=0.1, class_weight='balanced', gamma=1)
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4730941704035874
ROC-AUC: 0.5
F1-score: 0.0


In [119]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, model, tree = True, grid_search=True, grid_parameters = grid_params)
# too bad

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=0.1)
Best parameters from grid search: {'learning_rate': 0.1, 'n_estimators': 50}
Accuracy: 0.49407114624505927
ROC-AUC: 0.49575053118360213
F1-score: 0.1232876712328767


In [120]:
model = AdaBoostClassifier(learning_rate=0.1, n_estimators=500)
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, model, tree = True)

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.5059288537549407
ROC-AUC: 0.5069053868266467
F1-score: 0.34554973821989526


In [121]:
# testing
model = model = AdaBoostClassifier(learning_rate=0.1, n_estimators=100)
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.4977578475336323
ROC-AUC: 0.5168700211757589
F1-score: 0.25333333333333335


In [122]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [123]:
# testing
model = model = GaussianNB()
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.4730941704035874
ROC-AUC: 0.5
F1-score: 0.0


In [124]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [125]:
# testing
model = model = LogisticRegression()
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# not bad

Accuracy: 0.4730941704035874
ROC-AUC: 0.5
F1-score: 0.0


In [126]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier()
Best parameters from grid search: {'n_neighbors': 5}
Accuracy: 0.5177865612648221
ROC-AUC: 0.5180602424696913
F1-score: 0.4830508474576271


In [127]:
# testing
model = model = KNeighborsClassifier(n_neighbors=5)
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# not bad

Accuracy: 0.5224215246636771
ROC-AUC: 0.5262377735202178
F1-score: 0.5011709601873536


# All sentiments with lags

In [128]:
df

,Date_15m,datetime,text,username,kk08_pos_score,kk08_score_diff,Elkulako_label,Sentiment_vader,pred,analysis,...,Return_sign,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3,Sentiment_vader_lag1,Sentiment_vader_lag2,Sentiment_vader_lag3,Elkulako_label_lag1,Elkulako_label_lag2,Elkulako_label_lag3
0,2022-01-01 00:15:00,"['2022-01-01 00:00:00', '2022-01-01 00:00:00',...",Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,CryptoNerdApp\nTrendSpider\nMadStudentScie1\nW...,0.816322,0.632644,0.342175,0.161173,11,4,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01 00:30:00,"['2022-01-01 00:15:01', '2022-01-01 00:15:05',...",The current price of Bitcoin is $46453.00.\nTh...,big_blockers\nbmurphypointman\nButterLemonBot\...,0.716851,0.433702,0.305638,0.136947,9,42,...,1,0.816322,NaN,NaN,0.161173,NaN,NaN,0.342175,NaN,NaN
2,2022-01-01 00:45:00,"['2022-01-01 00:30:00', '2022-01-01 00:30:01',...",“Here's to a bright New Year and a fond farewe...,bitcoiva\ncryptozone007\nlinuxadvocate\nbakedn...,0.725069,0.450138,0.438272,0.172915,9,15,...,1,0.716851,0.816322,NaN,0.136947,0.161173,NaN,0.305638,0.342175,NaN
3,2022-01-01 01:00:00,"['2022-01-01 00:45:02', '2022-01-01 00:45:04',...","#Bitcoin $BTC \nPrice: $46,630.10\n\nAreas of ...",DCA_Cryptoz\nbidcoindad\ncoinok\nbmurphypointm...,0.756357,0.512714,0.381818,0.178866,5,11,...,1,0.725069,0.716851,0.816322,0.172915,0.136947,0.161173,0.438272,0.305638,0.342175
4,2022-01-01 01:15:00,"['2022-01-01 01:00:00', '2022-01-01 01:00:00',...",Today's Cryptocurrency Fear And Greed Index: 2...,FearAndGreedBot\nOfficialOverbit\nBTC_AI_bot\n...,0.770306,0.540612,0.343195,0.143037,6,13,...,1,0.756357,0.725069,0.716851,0.178866,0.172915,0.136947,0.381818,0.438272,0.305638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,2022-01-31 23:00:00,"['2022-01-31 22:45:00', '2022-01-31 22:45:01',...",NFL Stars and Politicians' Bet in Bitcoin Turn...,got_eth\nzero_2_nothing\nNewsCryptoBot\nJamesW...,0.771425,0.542850,0.209877,0.167436,1,4,...,1,0.780470,0.790736,0.812888,0.171507,0.120725,0.163388,0.202857,0.214076,0.241458
2972,2022-01-31 23:15:00,"['2022-01-31 23:00:00', '2022-01-31 23:00:00',...",Top #Cryptocurrency RANK DOWN (#crypto #bitcoi...,CryptoTopCharts\nBTC_AI_bot\ncryptonewsleak\nt...,0.779117,0.558234,0.263158,0.161682,4,4,...,0,0.771425,0.780470,0.790736,0.167436,0.171507,0.120725,0.209877,0.202857,0.214076
2973,2022-01-31 23:30:00,"['2022-01-31 23:15:00', '2022-01-31 23:15:00',...",Why are people buying #Bitcoin? Why are they a...,ForwardGroupMS\nThomasBakerMS\nbig_blockers\nb...,0.783670,0.567341,0.278592,0.161257,5,6,...,0,0.779117,0.771425,0.780470,0.161682,0.167436,0.171507,0.263158,0.209877,0.202857
2974,2022-01-31 23:45:00,"['2022-01-31 23:30:01', '2022-01-31 23:30:02',...","Bitcoin On Its Way To $500,000? Anthony Scaram...",OutlawBuckeye\nSifHomes\nitvc_\nRaulGlzC\nTheM...,0.769362,0.538723,0.240642,0.110640,3,4,...,1,0.783670,0.779117,0.771425,0.161257,0.161682,0.167436,0.278592,0.263158,0.209877


In [129]:
pd.concat([df_x_2, df_x_4, df_x_6], axis =1)

,kk08_pos_score,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3,Sentiment_vader,Sentiment_vader_lag1,Sentiment_vader_lag2,Sentiment_vader_lag3,Elkulako_label,Elkulako_label_lag1,Elkulako_label_lag2,Elkulako_label_lag3
0,0.756357,0.725069,0.716851,0.816322,0.178866,0.172915,0.136947,0.161173,0.381818,0.438272,0.305638,0.342175
1,0.770306,0.756357,0.725069,0.716851,0.143037,0.178866,0.172915,0.136947,0.343195,0.381818,0.438272,0.305638
2,0.765516,0.770306,0.756357,0.725069,0.168293,0.143037,0.178866,0.172915,0.358209,0.343195,0.381818,0.438272
3,0.768058,0.765516,0.770306,0.756357,0.201061,0.168293,0.143037,0.178866,0.323529,0.358209,0.343195,0.381818
4,0.805194,0.768058,0.765516,0.770306,0.199579,0.201061,0.168293,0.143037,0.345455,0.323529,0.358209,0.343195
...,...,...,...,...,...,...,...,...,...,...,...,...
2968,0.771425,0.780470,0.790736,0.812888,0.167436,0.171507,0.120725,0.163388,0.209877,0.202857,0.214076,0.241458
2969,0.779117,0.771425,0.780470,0.790736,0.161682,0.167436,0.171507,0.120725,0.263158,0.209877,0.202857,0.214076
2970,0.783670,0.779117,0.771425,0.780470,0.161257,0.161682,0.167436,0.171507,0.278592,0.263158,0.209877,0.202857
2971,0.769362,0.783670,0.779117,0.771425,0.110640,0.161257,0.161682,0.167436,0.240642,0.278592,0.263158,0.209877


In [130]:
df_x_all = pd.concat([df_x_2, df_x_4, df_x_6], axis =1)
df_y_all = df["Return_sign"].iloc[3:].reset_index(drop=True)

In [131]:
df_x_all

,kk08_pos_score,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3,Sentiment_vader,Sentiment_vader_lag1,Sentiment_vader_lag2,Sentiment_vader_lag3,Elkulako_label,Elkulako_label_lag1,Elkulako_label_lag2,Elkulako_label_lag3
0,0.756357,0.725069,0.716851,0.816322,0.178866,0.172915,0.136947,0.161173,0.381818,0.438272,0.305638,0.342175
1,0.770306,0.756357,0.725069,0.716851,0.143037,0.178866,0.172915,0.136947,0.343195,0.381818,0.438272,0.305638
2,0.765516,0.770306,0.756357,0.725069,0.168293,0.143037,0.178866,0.172915,0.358209,0.343195,0.381818,0.438272
3,0.768058,0.765516,0.770306,0.756357,0.201061,0.168293,0.143037,0.178866,0.323529,0.358209,0.343195,0.381818
4,0.805194,0.768058,0.765516,0.770306,0.199579,0.201061,0.168293,0.143037,0.345455,0.323529,0.358209,0.343195
...,...,...,...,...,...,...,...,...,...,...,...,...
2968,0.771425,0.780470,0.790736,0.812888,0.167436,0.171507,0.120725,0.163388,0.209877,0.202857,0.214076,0.241458
2969,0.779117,0.771425,0.780470,0.790736,0.161682,0.167436,0.171507,0.120725,0.263158,0.209877,0.202857,0.214076
2970,0.783670,0.779117,0.771425,0.780470,0.161257,0.161682,0.167436,0.171507,0.278592,0.263158,0.209877,0.202857
2971,0.769362,0.783670,0.779117,0.771425,0.110640,0.161257,0.161682,0.167436,0.240642,0.278592,0.263158,0.209877


In [132]:
df_y_6

0       1
1       1
2       0
3       1
4       1
       ..
2968    1
2969    0
2970    0
2971    1
2972    1
Name: Return_sign, Length: 2973, dtype: int64

In [133]:
train_share = 0.85
df_x_all_train = df_x_all.iloc[:int(len(df_x_all)*train_share)].reset_index(drop=True)
df_y_all_train = df_y_all.iloc[:int(len(df_x_all)*train_share)].reset_index(drop=True)

df_x_all_test = df_x_all.iloc[int(len(df_x_all)*train_share):].reset_index(drop=True)
df_y_all_test = df_y_all.iloc[int(len(df_x_all)*train_share):].reset_index(drop=True)

In [134]:
sum(df_y_all_test)/len(df_y_all_test)

0.5269058295964125

In [135]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_all_train,df_y_all_train,dummy_model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [136]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {'n_estimators': [5, 10, 20, 50, 100, 300, 500]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_all_train,df_y_all_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(n_estimators=10, random_state=17)
Best parameters from grid search: {'n_estimators': 10}
Accuracy: 0.4901185770750988
ROC-AUC: 0.49106361704786905
F1-score: 0.3316062176165803


In [137]:
# testing
model = RandomForestClassifier(n_estimators=10,random_state=17)
model.fit(df_x_all_train, df_y_all_train)
predictions = model.predict(df_x_all_test)
accuracy = accuracy_score(df_y_all_test, predictions)
roc_auc = roc_auc_score(df_y_all_test, predictions)
f1 = f1_score(df_y_all_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.484304932735426
ROC-AUC: 0.4907935867701926
F1-score: 0.4306930693069307


In [138]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [100, 300, 500],"max_depth": [1,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_all_train,df_y_all_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 1, 'n_estimators': 100}
Accuracy: 0.4782608695652174
ROC-AUC: 0.47919010123734535
F1-score: 0.31958762886597936


In [139]:
# testing
model = XGBClassifier(random_state=17, max_depth = 1, n_estimators= 100)
model.fit(df_x_all_train, df_y_all_train)
predictions = model.predict(df_x_all_test)
accuracy = accuracy_score(df_y_all_test, predictions)
roc_auc = roc_auc_score(df_y_all_test, predictions)
f1 = f1_score(df_y_all_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# not bad

Accuracy: 0.5134529147982063
ROC-AUC: 0.5184531612382777
F1-score: 0.47961630695443647


In [140]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_all_train,df_y_all_train, svm_model, grid_search=True, grid_parameters=grid_params)
# too bad

SVC(C=1, class_weight='balanced', gamma=1, kernel='poly')
Best parameters from grid search: {'C': 1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'poly'}
Accuracy: 0.4782608695652174
ROC-AUC: 0.47890888638920126
F1-score: 0.37735849056603776


In [141]:
# testing
model = SVC(C=1, class_weight='balanced', gamma=1)
model.fit(df_x_all_train, df_y_all_train)
predictions = model.predict(df_x_all_test)
accuracy = accuracy_score(df_y_all_test, predictions)
roc_auc = roc_auc_score(df_y_all_test, predictions)
f1 = f1_score(df_y_all_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.46860986547085204
ROC-AUC: 0.4691237269335484
F1-score: 0.4768211920529801


In [142]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_all_train,df_y_all_train, model, tree = True, grid_search=True, grid_parameters = grid_params)


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=0.3)
Best parameters from grid search: {'learning_rate': 0.3, 'n_estimators': 50}
Accuracy: 0.5059288537549407
ROC-AUC: 0.507092863392076
F1-score: 0.3016759776536313


In [143]:
# testing
model = model = AdaBoostClassifier(learning_rate=0.3, n_estimators=50)
model.fit(df_x_all_train, df_y_all_train)
predictions = model.predict(df_x_all_test)
accuracy = accuracy_score(df_y_all_test, predictions)
roc_auc = roc_auc_score(df_y_all_test, predictions)
f1 = f1_score(df_y_all_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.5
ROC-AUC: 0.502057073711808
F1-score: 0.4943310657596372


In [144]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_all_train,df_y_all_train, model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [145]:
# testing
model = model = GaussianNB()
model.fit(df_x_all_train, df_y_all_train)
predictions = model.predict(df_x_all_test)
accuracy = accuracy_score(df_y_all_test, predictions)
roc_auc = roc_auc_score(df_y_all_test, predictions)
f1 = f1_score(df_y_all_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.47533632286995514
ROC-AUC: 0.502127659574468
F1-score: 0.00847457627118644


In [146]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_all_train,df_y_all_train, model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier()
Best parameters from grid search: {'n_neighbors': 5}
Accuracy: 0.45849802371541504
ROC-AUC: 0.4589113860767404
F1-score: 0.3964757709251101


In [147]:
# testing
model = model = KNeighborsClassifier(n_neighbors=5)
model.fit(df_x_all_train, df_y_all_train)
predictions = model.predict(df_x_all_test)
accuracy = accuracy_score(df_y_all_test, predictions)
roc_auc = roc_auc_score(df_y_all_test, predictions)
f1 = f1_score(df_y_all_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.5067264573991032
ROC-AUC: 0.5069880004033477
F1-score: 0.5175438596491229


# With dummies

In [148]:
df.columns

Index(['Date_15m', 'datetime', 'text', 'username', 'kk08_pos_score',
       'kk08_score_diff', 'Elkulako_label', 'Sentiment_vader', 'pred',
       'analysis', 'short', 'long', 'buy', 'sell', 'bull', 'bear', 'bankrupt',
       'fraud', 'cbdc', 'record', 'crisis', 'collapse', 'ATH', 'grow', 'rise',
       'fall', 'date', 'open', 'close', 'Return', 'Return_sign',
       'kk08_pos_score_lag1', 'kk08_pos_score_lag2', 'kk08_pos_score_lag3',
       'Sentiment_vader_lag1', 'Sentiment_vader_lag2', 'Sentiment_vader_lag3',
       'Elkulako_label_lag1', 'Elkulako_label_lag2', 'Elkulako_label_lag3'],
      dtype='object')

In [149]:
df_x_dum = df[['kk08_pos_score', 'Elkulako_label', 'Sentiment_vader', 'pred',
       'analysis', 'short', 'long', 'buy', 'sell', 'bull', 'bear', 'bankrupt',
       'fraud', 'cbdc', 'record', 'crisis', 'collapse', 'ATH', 'grow', 'rise',
       'fall']]
df_y = df["Return_sign"]

In [150]:
df_x_dum

,kk08_pos_score,Elkulako_label,Sentiment_vader,pred,analysis,short,long,buy,sell,bull,...,bankrupt,fraud,cbdc,record,crisis,collapse,ATH,grow,rise,fall
0,0.816322,0.342175,0.161173,11,4,4,7,27,18,4,...,0,0,0,0,0,0,9,6,7,4
1,0.716851,0.305638,0.136947,9,42,41,13,22,8,10,...,0,1,0,1,0,1,8,1,6,38
2,0.725069,0.438272,0.172915,9,15,17,6,22,7,8,...,0,0,0,1,0,0,6,2,5,17
3,0.756357,0.381818,0.178866,5,11,9,5,16,4,1,...,0,0,1,2,0,1,5,1,2,6
4,0.770306,0.343195,0.143037,6,13,12,2,26,9,10,...,0,0,1,1,0,0,6,8,4,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,0.771425,0.209877,0.167436,1,4,5,8,25,12,7,...,0,0,1,1,0,1,10,3,3,8
2972,0.779117,0.263158,0.161682,4,4,6,10,25,7,10,...,0,0,0,0,0,0,5,5,5,3
2973,0.783670,0.278592,0.161257,5,6,5,11,21,7,11,...,0,1,0,1,0,0,6,5,5,3
2974,0.769362,0.240642,0.110640,3,4,6,15,24,17,11,...,0,2,0,1,17,1,4,1,8,6


In [151]:
train_share = 0.85
df_x_dum_train = df_x_dum.iloc[:int(len(df_x_dum)*train_share)].reset_index(drop=True)
df_y_train = df_y.iloc[:int(len(df_y)*train_share)].reset_index(drop=True)

df_x_dum_test = df_x_dum.iloc[int(len(df_x_dum)*train_share):].reset_index(drop=True)
df_y_test = df_y.iloc[int(len(df_y)*train_share):].reset_index(drop=True)

In [152]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_dum_train,df_y_train,dummy_model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [153]:
# testing
model = DummyClassifier(strategy='most_frequent')

model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4720357941834452
ROC-AUC: 0.5
F1-score: 0.0


In [154]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {'n_estimators': [50,100, 300 ,500, 700]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(n_estimators=500, random_state=17)
Best parameters from grid search: {'n_estimators': 500}
Accuracy: 0.5098814229249012
ROC-AUC: 0.5107486564179478
F1-score: 0.37373737373737376


In [155]:
# testing
model = RandomForestClassifier(n_estimators=500,random_state=17)
model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.4697986577181208
ROC-AUC: 0.47830146999759016
F1-score: 0.3938618925831202


In [156]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [50, 100, 300, 500, 1000],"max_depth": [1,2,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 3, 'n_estimators': 1000}
Accuracy: 0.48221343873517786
ROC-AUC: 0.48259592550931124
F1-score: 0.4279475982532751


In [157]:
# testing
model = XGBClassifier(random_state=17, max_depth = 3, n_estimators = 1000)
model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

Accuracy: 0.5123042505592841
ROC-AUC: 0.5140372720700457
F1-score: 0.5112107623318386


In [158]:
from sklearn.svm import SVC
# SVM
svm_model = SVC(kernel="poly")
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, svm_model)

Accuracy: 0.4980237154150198
ROC-AUC: 0.5
F1-score: 0.0


In [159]:
# testing
model = svm_model = SVC(kernel="poly")
model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4720357941834452
ROC-AUC: 0.5
F1-score: 0.0


In [160]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
ada_model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, ada_model, tree = True, grid_search=True, grid_parameters = grid_params)


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=1, n_estimators=300)
Best parameters from grid search: {'learning_rate': 1, 'n_estimators': 300}
Accuracy: 0.49407114624505927
ROC-AUC: 0.4945944256967879
F1-score: 0.41818181818181815


In [161]:
# testing
model = AdaBoostClassifier(learning_rate=0.1, n_estimators=300)
model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.465324384787472
ROC-AUC: 0.4723070126114547
F1-score: 0.40694789081885857


In [162]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, nb_model)


Accuracy: 0.4980237154150198
ROC-AUC: 0.49993750781152363
F1-score: 0.030534351145038167


In [163]:
# testing
model = GaussianNB()
model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

Accuracy: 0.4720357941834452
ROC-AUC: 0.4994979516427022
F1-score: 0.016666666666666666


In [164]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(max_iter=10000)
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, lr_model)


Accuracy: 0.5019762845849802
ROC-AUC: 0.5031871016122985
F1-score: 0.2840909090909091


In [165]:
# testing
model = LogisticRegression(max_iter=10000)

model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4742729306487696
ROC-AUC: 0.49283074945778776
F1-score: 0.24437299035369775


In [166]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
knn_model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, knn_model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier(n_neighbors=2)
Best parameters from grid search: {'n_neighbors': 2}
Accuracy: 0.45454545454545453
ROC-AUC: 0.4557242844644419
F1-score: 0.2247191011235955


In [167]:
# testing
model = KNeighborsClassifier(n_neighbors=2)
model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

Accuracy: 0.4519015659955257
ROC-AUC: 0.4688830428146839
F1-score: 0.24148606811145512
